In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from remote_api_ggg import remote_api
from datetime import datetime
from dateutil import tz
import pytz
import time
import sys

#path = "/var/www/wb_sanisidro/"
path=''
tokens_users={
    #'SanIsidro23':"c9c7911221d3b371a04dc0d5fea633b31C1F99DD55929821A79B0B469EC7FBF31CDC041C",
    '4TrackDemo':"c9c7911221d3b371a04dc0d5fea633b3C330C2ABDF6EC887B73957D9ED80C67A302674A9"
}

inicio = int(datetime(2024,3,1,0,0,0,tzinfo=tz.gettz('America/Monterrey')).timestamp())
fin = int(datetime(2024,3,31,23,59,59,tzinfo=tz.gettz('America/Monterrey')).timestamp())
data = pd.DataFrame()

In [19]:
for i in tokens_users:
    api = remote_api(tokens_users[i])
    sid, uid = api.autorizacion()
    r = api.obtencion_unidades_ids(sid,'Mazda-CX-50',i,4611686018427387903).json()['items']
    #print(r)
    for i in r:
        try:
            m = api.carga_mensajes(i['id'],sid,inicio,fin).json()

            if m['count'] > 0:
                print(i['id'])
                d = pd.DataFrame(m['messages'])
                pos = pd.DataFrame(list(d['pos']))
                p = pd.DataFrame(list(d['p']))
                d[['position.latitude','position.longitude']] = pos[['y','x']]
                d['ignition'] = p['io_239']

            #print(m['messages'][0])
            api.limpiar_mensajes(sid)
            print(f"ok con {i['nm']}")

        except:
            print('problemas con la unidad {}'.format(i['nm']))
        

d.rename(columns={'t':'timestamp','rt':'timestamp registered'},inplace=True)
d = d.sort_values(by='timestamp',ascending=True)
d['diferencia'] = d['timestamp registered'] - d['timestamp']
d['conexión'] = 'Conectado'
d.loc[((d['diferencia']>=2*60) & (d['diferencia']<10*60)),'conexión']='Desconexión 2-10 min'
d.loc[((d['diferencia']>=10*60) & (d['diferencia']<30*60)),'conexión']='Desconexión 10-30 min'
d.loc[((d['diferencia']>=30*60)),'conexión']='Desconexión mas de 30 min'

d['fecha hora'] = pd.to_datetime(d['timestamp'],unit='s',utc=True)
d['fecha hora'] = d['fecha hora'].dt.tz_convert(tz=tz.gettz("America/Monterrey"))
d['ignition'] = d['ignition'].map({0.0:'Apagado',1.0:'Encendido'})

on = d[d['ignition']==1][['timestamp','timestamp registered','position.latitude','position.longitude','ignition','diferencia','conexión','fecha hora']]
off = d[d['ignition']==0][['timestamp','timestamp registered','position.latitude','position.longitude','ignition','diferencia','conexión','fecha hora']]

401331049
ok con Mazda-CX-50


In [51]:
import plotly.express as px
barras = d[['conexión','ignition']].value_counts().reset_index(drop=False)
fig = px.bar(barras, x='conexión', y='count', color='ignition', text='count',
             title='Total de registros apagado {}'.format(len(d)),
             labels={"count":'Número de registros'},barmode='group')
fig.show()

In [55]:
import plotly.io as pio
on['horas desconectado'] = on['diferencia']/(60*60)
fig = px.scatter_mapbox(on,title='''Conexiones''',
                        lat='position.latitude', lon='position.longitude',color='horas desconectado',hover_name='fecha hora',
                        hover_data=['horas desconectado'],color_continuous_scale='Bluered_r')
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
#fig.update_geos(lataxis_range=[26,25], lonaxis_range=[-100, -100.5])
#fig.update_traces(marker={"size": 50, "symbol": ["*"]})
pio.write_image(fig, 'mapa.png',height=400,width=1000,scale=6)
fig.show()

In [58]:
his = on[on['diferencia']>=60*30]
his['diferencia'] = his['diferencia']/(60*60)
import plotly.express as px
df = px.data.tips()
fig = px.histogram(his, x="diferencia",nbins=30,text_auto=True,
                   title='Distribución de horas desconectadas',
                   labels={'diferencia':'Horas desconectado'})
fig.show()

/tmp/ipykernel_363/508183654.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

